# This notebook (future script) is the basis of processing ERA5 files and calculating environmental variables for creating a ML TCGI. 

In [1]:
# Import libraries
import xarray as xr
import os
import numpy as np
import datetime as dt
import copy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import time
from functools import partial

# Begin by sorting through which ERA5 files are desired for each variable, including the date intervals

In [2]:
def era_5_datestrings(data_interval): # days
    months_from_dir = sorted(os.listdir('/glade/collections/rda/data/ds633.0/e5.oper.an.pl'))
    first_month_str = months_from_dir[0]
    first_month_dt = dt.datetime.strptime(first_month_str, '%Y%m')
    last_month_str = months_from_dir[-1]
    last_month_dt = dt.datetime.strptime(last_month_str, '%Y%m')
    # We process all data except for the most current year (so we have an entire year for every year we look at)
    end_year = last_month_dt.year

    date_range_list = []
    current_dt = copy.deepcopy(first_month_dt)
    date_range_list.append(current_dt)
    # Create a list of datetimes from start to last full year

    while current_dt.year < end_year: 
        current_dt = current_dt + dt.timedelta(days=1)
        if current_dt.year < end_year:    
            date_range_list.append(current_dt)
    
    return date_range_list

# 1) Absolute Vorticity

In [3]:
def generate_pathstrs(date_range_list,variable_id):
# Create all path strings

    all_path_strs = []
    for current_date in date_range_list:
        if current_date.month < 10:
            current_month = '0'+str(current_date.month)
        else:
            current_month = str(current_date.month)

        if current_date.day < 10:
            current_day_num = '0' + str(current_date.day)
        else:
            current_day_num = str(current_date.day)

        path_str = '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/'+str(current_date.year)+current_month+'/e5.oper.an.pl.128_' + variable_id + '.ll025sc.' + str(current_date.year) + current_month +current_day_num + '00_'+ str(current_date.year) + current_month +current_day_num + '23.nc' 
        all_path_strs.append(path_str)
    
    return all_path_strs

In [4]:
data_interval = 7
date_range_list = era_5_datestrings(data_interval)
variable_id = '138_vo' # for relative vorticity

all_path_strs = generate_pathstrs(date_range_list,variable_id)

In [7]:
# Open only pressure level 850 hPa
def _preprocess(x,level):
    return x.sel(level=level)

start = time.time()

level = 850 # hPa
partial_func = partial(_preprocess, level=level)

# Load weekly data
current_path_strs = all_path_strs[0:7]
datasets = xr.open_mfdataset(current_path_strs,preprocess=partial_func, parallel=True)['VO'].drop('level')
# Create array Coriolis Parameter
omega = 7.292 * 10 **-5 # 1/s
f = 2*omega*np.sin(np.deg2rad(datasets.latitude))

# Calculate Absolute Vorticity
abs_vort = datasets + f

# Calculate interval mean
mean_abs_vort_850 = abs_vort.mean('time')
mean_abs_vort_850 = mean_abs_vort_850.assign_coords({"beg":np.asarray(abs_vort['time'][0])})
mean_abs_vort_850 = mean_abs_vort_850.assign_coords({"end":np.asarray(abs_vort['time'][-1])})

# Save absolute vorticity
path = "/glade/scratch/acheung/abs_vort"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
# Create a new directory because it does not exist
    os.makedirs(path) 


variable_file_name_base = current_path_strs[0][57:90] + str(data_interval) + 'd_mean.'
variable_file_name_start_time = current_path_strs[0][90:101]
variable_file_name_end_time = current_path_strs[-1][101:]

var_file_name_full = variable_file_name_base + variable_file_name_start_time + variable_file_name_end_time


mean_abs_vort_850.to_dataset(name='Absolute Vorticity').to_netcdf('/glade/scratch/acheung/abs_vort/'+var_file_name_full)

# end time
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")

Runtime of the program is 147.0346736907959


In [ ]:
plt.figure(figsize=(10,7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.set_extent([-100, 0, 0, 80])
letsee['Absolute Vorticity'].plot(vmin=0,vmax=0.00015)


In [46]:
all_path_strs

['/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010100_1940010123.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010200_1940010223.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010300_1940010323.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010400_1940010423.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010500_1940010523.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010600_1940010623.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010700_1940010723.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an.pl/194001/e5.oper.an.pl.128_138_vo.ll025sc.1940010800_1940010823.nc',
 '/glade/collections/rda/data/ds633.0/e5.oper.an